<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/DC_OPF_robust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code for the robust DC optimal power flow.

# Requirements

In [ ]:
!pip install -q gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

     |████████████████████████████████| 11.5 MB 4.9 MB/s 


# Data

In [ ]:
# generation data
gen = pd.DataFrame({        
       'bus':       [0,    1, 2],
       'cost':      [20,   30, 0], 
       'cost_up':   [30,   35, 0], 
       'cost_do':   [10,   25, 0], 
       'pmin':      [0,   0, 0], 
       'pmax':      [100,  50, 0]})

lin = pd.DataFrame({
       'line': [0,   1,   2], 
       'from': [0,   0,   1],
       'to':   [1,   2,   2], 
       'b':    [1,  1,  1], 
       'cap':  [60, 60, 60]})

# Number of elements in the system
nbus = max(max(lin['from']),max(lin['to']))+1
ngen = nbus
nlin = len(lin)

# Power Transfer Distribution Factor
A_max = np.zeros((nlin,nbus))
for l in range(nlin): 
    A_max[l,lin.loc[l,'from']] = 1
    A_max[l,lin.loc[l,'to']]   = -1 
node_ref = 0
A_m  = np.delete(A_max,node_ref,axis=1)
X_m  = np.diag(1/lin['b'].values)
B_m  = np.linalg.multi_dot([A_m.T,np.linalg.inv(X_m),A_m])
ptdf = np.linalg.multi_dot([np.linalg.inv(X_m),A_m,np.linalg.inv(B_m)])
PTDF = np.round(np.insert(ptdf,node_ref,np.zeros(nlin),axis=1),5)

# State of the system
demand = np.array([0,0,90])
var = np.array([0,0,20])      
scen = np.zeros((len(demand),11))
scen[2,:] = np.array([-1,-0.8,-0.6,-0.4,-0.2,0,0.2,0.4,0.6,0.8,1])
scen20    = 20*scen
demand_scenarios = np.reshape(demand,(3,1)) + scen20

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ 70.  74.  78.  82.  86.  90.  94.  98. 102. 106. 110.]]


# Robust DC-OPF

\begin{align}
\underset{p_n}{\min} \quad & \sum_n c_n p_n  \\
\text{s.t.} \quad & \sum_n p_n \geq \sum_n (\hat{d}_n + e_n) \\
& \underline{p}_n \leq p_n \leq \overline{p}_n, \quad \forall n\\
& -\underline{f}_l \leq \sum_n B_{ln}\left(p_n  -\hat{d}_n \right) - \sum_n e_n|B_{ln} |, \quad \forall l \\
&  \sum_n B_{ln} \left( p_n -\hat{d}_n \right) + \sum_n e_n|B_{ln}| \leq \overline{f}_l, \quad \forall l. 
\end{align}

In [ ]:
def dc_opf_robust(demand,var):
  # model
  m = gp.Model() 
  # variables
  power = m.addMVar(ngen,name='gen')
  # objective function
  m.setObjective(gen['cost'].values @ power, GRB.MINIMIZE)
  # constraints
  m.addConstr(power.sum() >= demand.sum() + var.sum())
  m.addConstr(power >= gen['pmin'].values)
  m.addConstr(power <= gen['pmax'].values)
  m.addConstr(PTDF @ power - PTDF @ demand - abs(PTDF) @ var >= -lin['cap'].values)
  m.addConstr(PTDF @ power - PTDF @ demand + abs(PTDF) @ var <=  lin['cap'].values)
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  
  # results
  print('########## Robust #############')
  print('cost =',round(m.ObjVal,1))
  print('dispatch =',np.round(power.X,2))
  return power.X


# Real time operation

\begin{align}
\underset{r^u_n\geq 0,r^d_n\geq 0}{\min} \quad & \sum_n c_n p^*_n + c^u_n r^u_n - c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n p^*_n+r^u_n-r^d_n = \sum_n \tilde{d}_n - e \eta_n\\
& \underline{p} \leq p^*_n + r^u_n-r^d_n \leq \overline{p}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(p^*_n + r^u_n-r^d_n -\tilde{d}_n) \leq \overline{f}_l, \quad \forall l \\
\end{align}

In [ ]:
def real_time(demand,dispatch):
  # model
  m = gp.Model()
  # variables
  res_up = m.addMVar(ngen)
  res_do = m.addMVar(ngen)
  # objective function
  m.setObjective(gen['cost'].values @ dispatch + gen['cost_up'].values @ res_up - gen['cost_do'].values @ res_do, GRB.MINIMIZE)
  # constraints
  m.addConstr(dispatch.sum() + res_up.sum() - res_do.sum() == demand.sum())
  m.addConstr(dispatch + res_up - res_do >= gen['pmin'].values)
  m.addConstr(dispatch + res_up - res_do <= gen['pmax'].values)
  m.addConstr(PTDF @ dispatch + PTDF @ res_up - PTDF @ res_do - PTDF @ demand >= -lin['cap'].values)
  m.addConstr(PTDF @ dispatch + PTDF @ res_up - PTDF @ res_do - PTDF @ demand <=  lin['cap'].values)
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  return m.ObjVal

In [ ]:
def evaluate_scenarios(dispatch,demand_scenarios):
  v_cost = []
  for s in range(demand_scenarios.shape[1]):
    obj = real_time(demand_scenarios[:,s],dispatch)
    v_cost.append(obj)    
  print('minimum cost =',round(np.min(v_cost),1))
  print('average cost =',round(np.mean(v_cost),1))
  print('maximum cost =',round(np.max(v_cost),1))

# Evaluate the robust DC-OPF

In [ ]:
gen_rb  = dc_opf_robust(demand,var)
evaluate_scenarios(np.round(gen_rb,2), demand_scenarios)

########## Robust #############
cost = 2600.0
dispatch = [70. 40.  0.]
minimum cost = 1600.0
average cost = 2100.0
maximum cost = 2600.0
